# 4D emittance measurement data analysis 

In [ ]:
import sys
import os
import copy
import importlib
from tqdm import trange, tqdm

import numpy as np
import pandas as pd
from scipy import optimize as opt
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import animation
import proplot as pplt
import seaborn as sns

sys.path.append('/Users/46h/Research/')
from accphys.tools import beam_analysis as ba
from accphys.tools import coupling as BL
from accphys.tools import plotting as myplt
from accphys.tools import utils
from accphys.tools.accphys_utils import V_matrix_4x4_uncoupled, phase_adv_matrix

from accphys.emittance_measurement_4D.analysis import reconstruct
from accphys.emittance_measurement_4D.analysis import to_mat, to_vec
from accphys.emittance_measurement_4D.plotting import reconstruction_lines

In [ ]:
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['grid.alpha'] = 0.04
pplt.rc['axes.grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['savefig.transparent'] = False
pplt.rc['savefig.dpi'] = 'figure'
pplt.rc['animation.html'] = 'jshtml'

In [ ]:
eps_labels = [r'$\varepsilon_x$', r'$\varepsilon_y$', 
              r'$\varepsilon_1$', r'$\varepsilon_2$']
savefig_kws = dict(facecolor='white', dpi=300)

## Load data 

In [ ]:
folder = './_saved/2021-09-26/setting1/ramp_turns/'

# We can restrict the number of measurements used if we want.
max_n_meas = 100 

# We can exclude certain wire-scanners if we want.
exclude = None

In [ ]:
utils.delete_files_not_folders('_output/figures/')

In [ ]:
def load_data_dict(filename, ndarray=True):
    dictionary = dict()
    file = open(filename, 'r')
    for line in file:
        tokens = line.rstrip().split(' ')
        ws_id = tokens[0]
        items = [float(token) for token in tokens[1:]]
        if ws_id not in dictionary:
            dictionary[ws_id] = []
        dictionary[ws_id].append(items)
    file.close()
    dictionary = utils.blacklist(dictionary, exclude)
    for ws_id in dictionary:
        if max_n_meas:
            dictionary[ws_id] = dictionary[ws_id][:max_n_meas]
        if ndarray:
            dictionary[ws_id] = np.array(dictionary[ws_id])
    return dictionary

In [ ]:
tmats_dict = load_data_dict(os.path.join(folder, 'info/transfer_mats.dat'), ndarray=False)
moments_dict = load_data_dict(os.path.join(folder, 'info/moments.dat'))
phases_dict = load_data_dict(os.path.join(folder, 'info/phase_adv.dat'))
pos_x_dict = load_data_dict(os.path.join(folder, 'info/pos_x.dat'))
pos_y_dict = load_data_dict(os.path.join(folder, 'info/pos_y.dat'))
pos_u_dict = load_data_dict(os.path.join(folder, 'info/pos_u.dat'))
raw_x_dict = load_data_dict(os.path.join(folder, 'info/raw_x.dat'))
raw_y_dict = load_data_dict(os.path.join(folder, 'info/raw_y.dat'))
raw_u_dict = load_data_dict(os.path.join(folder, 'info/raw_u.dat'))

ws_ids = sorted(list(tmats_dict))
n_meas = len(tmats_dict[ws_ids[0]])
meas_indices = list(range(n_meas))

# Convert from 16 element lists to 4x4 arrays for transfer matrices.
for ws_id in ws_ids:
    for i in range(n_meas):
        tmats_dict[ws_id][i] = np.reshape(tmats_dict[ws_id][i], (4, 4))

In [ ]:
fig, axes = pplt.subplots(nrows=3, figsize=(4.25, 4.5), spany=False, wspace=1, aligny=True)
plot_kws = dict(marker='.')
handles = []
for ws_id in ws_ids:
    moments = moments_dict[ws_id]
    axes[0].plot(moments[:, 0], **plot_kws)
    axes[1].plot(moments[:, 1], **plot_kws)
    h = axes[2].plot(moments[:, 2] / np.sqrt(moments[:, 0] * moments[:, 1]), **plot_kws)
    handles.append(h)
axes[0].format(ylabel=r'$\langle{x^2}\rangle$ [mm$^2$]')
axes[1].format(ylabel=r'$\langle{y^2}\rangle$ [mm$^2$]')
axes[2].format(ylabel=r'$\langle{xy}\rangle$ [mm$^2$]')
for ax in axes:
    ax.grid(axis='y', alpha=0.1)
axes.format(xlabel='Measurement index', xtickminor=False, xticks=meas_indices)
axes[0].legend(handles, labels=ws_ids, ncols=1, loc='r', fontsize='small')
axes[0].set_title('Measured moments', fontsize='medium')
plt.savefig('_output/figures/moments.png', **savefig_kws)

In [ ]:
fig, axes = pplt.subplots(nrows=3, figsize=(4.25, 4.5), spany=False, wspace=1, aligny=True)
handles = []
for ws_id in ws_ids:
    moments = moments_dict[ws_id]
    axes[0].plot(np.sqrt(moments[:, 0]), **plot_kws)
    axes[1].plot(np.sqrt(moments[:, 1]), **plot_kws)
    h = axes[2].plot(moments[:, 2] / np.sqrt(moments[:, 0] * moments[:, 1]), **plot_kws)
    handles.append(h)
axes[0].format(ylabel=r'$\sqrt{\langle{x^2}\rangle}$ [mm$^2$]')
axes[1].format(ylabel=r'$\sqrt{\langle{y^2}\rangle}$ [mm$^2$]')
axes[2].format(ylabel=r'x-y corr. coeff.')
for ax in axes:
    ax.grid(axis='y', alpha=0.1)
axes.format(xlabel='Measurement index', xtickminor=False, xticks=meas_indices)
axes[0].legend(handles, labels=ws_ids, ncols=1, loc='r', fontsize='small')
axes[0].set_title('Measured rms size and correlation', fontsize='medium')
plt.savefig('_output/figures/size_corr.png', **savefig_kws)

In [ ]:
fig, axes = pplt.subplots(ncols=2, figsize=(6, 2.5))
handles = []
for ws_id in ws_ids:
    mux, muy = phases_dict[ws_id].T
    axes[0].plot(mux, **plot_kws)
    h = axes[1].plot(muy, **plot_kws)
    handles.append(h)
axes.format(xlabel='Measurement index', ylabel='[rad]', ylim=(0, 2 * np.pi),
            xtickminor=False, xticks=range(n_meas))
axes[0].set_title('Horizontal phase adv.')
axes[1].set_title('Vertical phase adv.')
axes[1].legend(handles, ws_ids, ncols=1, loc=(1.02, 0), fontsize='small')
for ax in axes:
    ax.grid(axis='y')
plt.savefig('_output/figures/phases.png', **savefig_kws)

In [ ]:
# fig, axes = pplt.subplots(ncols=3)
# colors = ['red8', 'blue8', 'green8']
# for i in range(len(ws_ids) - 1):
#     diffs = phases_dict[ws_ids[i + 1]] - phases_dict[ws_ids[i]]
#     for j in range(len(diffs)):
#         for k in range(2):
#             if diffs[j, k] < 0:
#                 diffs[j, k] += 2 * np.pi
#     axes[0].plot(np.degrees(diffs[:, 0]), color=colors[i], marker='.')
#     axes[1].plot(np.degrees(diffs[:, 1]), color=colors[i], marker='.')
#     axes[2].plot(np.degrees(np.abs(diffs[:, 0] - diffs[:, 1])), color=colors[i], marker='.')
# for ax in axes:
#     ax.grid(axis='y')
# labels = [r'{} $\rightarrow$ {}'.format(ws_ids[i], ws_ids[i + 1]) for i in range(len(ws_ids) - 1)]
# axes[2].legend(labels=labels, ncol=1, loc=(1.02, 0))
# axes[0].set_title(r'Horizontal: $\Delta\mu_x$')
# axes[1].set_title(r'Vertical: $\Delta\mu_y$')
# axes[2].set_title(r'Difference: $\left| \Delta\mu_x - \Delta\mu_y \right|$')
# axes.format(ylabel='Gap', yformatter='deg', suptitle='Wire-scanner phase spacing',
#             xlabel='Measurement index', xtickminor=False, xticks=range(n_meas))
# plt.savefig('_output/figures/phase_gaps.png', **savefig_kws)

In [ ]:
fig, axes = pplt.subplots(ncols=4, figsize=(8, 2))
for ax, ws_id in zip(axes, ws_ids):
    ax.set_title(ws_id)
    for sig_xx, sig_yy, sig_xy in moments_dict[ws_id]:
        angle = -0.5 * np.arctan2(2*sig_xy, sig_xx-sig_yy)
        sn, cs = np.sin(angle), np.cos(angle)
        c1 = np.sqrt(abs(sig_xx*cs**2 + sig_yy*sn**2 - 2*sig_xy*sn*cs))
        c2 = np.sqrt(abs(sig_xx*sn**2 + sig_yy*cs**2 + 2*sig_xy*sn*cs))
        myplt.ellipse(ax, c1, c2, angle)
axes.format(xlim=(-30, 30), ylim=(-30, 30), xlabel='x [mm]', ylabel='y [mm]',
            suptitle='Measured rms ellipses in x-y plane')
plt.savefig('_output/figures/corr.png', **savefig_kws)

In [ ]:
plot_kws = dict(marker='.', ms=0)

for same_ylim in [False, True]:

    # Get global maximum signal height.
    ymax = 0.
    for meas_index in range(n_meas):
        for ws_id in ws_ids:
            fx = raw_x_dict[ws_id][meas_index]
            fy = raw_y_dict[ws_id][meas_index]
            fu = raw_u_dict[ws_id][meas_index]
            ymax = max(ymax, max(np.max(fx), np.max(fy), np.max(fu)))

    for meas_index in range(n_meas):
        fig, axes = pplt.subplots(ncols=3, figsize=(8, 1.5), sharex=False)
        handles = []
        for ws_id in ws_ids:
            axes[0].plot(pos_x_dict[ws_id][meas_index], raw_x_dict[ws_id][meas_index], **plot_kws)
            axes[1].plot(pos_y_dict[ws_id][meas_index], raw_y_dict[ws_id][meas_index], **plot_kws)
            g = axes[2].plot(pos_u_dict[ws_id][meas_index], raw_u_dict[ws_id][meas_index], **plot_kws)
            handles.append(g)
        axes[0].format(xlabel='x [mm]', title='Horizontal')
        axes[1].format(xlabel='y [mm]', title='Vertical')
        axes[2].format(xlabel='u [mm]', title='Diagonal')
        if same_ylim:
            ymin = axes[0].get_ylim()[0]
            axes.format(ylim=(ymin, ymax))
        for ax in axes:
            ax.grid(axis='y')
        axes[2].legend(handles, labels=ws_ids, ncols=1, loc='r', fontsize=6.5)
        
        if same_ylim:
            figname = '_output/figures/profiles_sameylim_{}.png'.format(meas_index)
        else:
            figname = '_output/figures/profiles_{}.png'.format(meas_index)
        plt.savefig(figname, **savefig_kws)
        plt.show()
        
    print()
    print()

In [ ]:
# emittances = []
# nkeep = list(range(1, n_meas))
# for j in nkeep:
#     tmats_list, moments_list = [], []
#     for ws_id in ws_ids:
#         tmats_list.extend(tmats_dict[ws_id][:j])
#         moments_list.extend(moments_dict[ws_id][:j])
#     Sigma = reconstruct(tmats_list, moments_list)
#     emittances.append(ba.emittances(Sigma))
# emittances = np.array(emittances)
    
# fig, ax = pplt.subplots()
# ax.plot(nkeep, np.sqrt(emittances[:, 0] * emittances[:, 1]) - np.sqrt(eps_x * eps_y), 
#         marker='.', color='red8')
# ax.plot(nkeep, np.sqrt(emittances[:, 2] * emittances[:, 3]) - np.sqrt(eps_1 * eps_2),
#         marker='.', color='blue8')
# ax.legend(labels=[r'$\sqrt{\varepsilon_x\varepsilon_y}$', r'$\sqrt{\varepsilon_1\varepsilon_2}$'], ncols=2)
# ax.format(xlabel='Number of measurements used', ylabel='Difference from full fit [mm mrad]')
# plt.savefig('_output/figures/nmeas.png', facecolor='white', dpi=300)

## Reconstruction

In [ ]:
tmats_list, moments_list = [], []
for ws_id in ws_ids:
    tmats_list.extend(tmats_dict[ws_id])
    moments_list.extend(moments_dict[ws_id])

In [ ]:
Sigma = reconstruct(tmats_list, moments_list, verbose=2)

In [ ]:
Corr = utils.cov2corr(Sigma)
alpha_x, alpha_y, beta_x, beta_y = ba.twiss2D(Sigma)
eps_x, eps_y, eps_1, eps_2 = ba.emittances(Sigma)
coupling_coeff = 1.0 - np.sqrt((eps_1 * eps_2) / (eps_x * eps_y))

print('Sigma =')
print(Sigma)
print('Corr =')
print(Corr)
print('eps_4D = {:.3f}'.format(np.sqrt(np.linalg.det(Sigma))))
print('eps_1, eps_2 = {:.3f}, {:.3f}'.format(eps_1, eps_2))
print('eps_x, eps_y = {:.3f}, {:.3f}'.format(eps_x, eps_y))
print('alpha_x, alpha_y = {:.3f}, {:.3f}'.format(alpha_x, alpha_y))
print('beta_x, beta_y = {:.3f}, {:.3f}'.format(beta_x, beta_y))
print('Coupling coefficient = {}'.format(coupling_coeff))

In [ ]:
norm = '2D'
V = np.identity(4)
if norm == '2D':
    alpha_x, alpha_y, beta_x, beta_y = ba.twiss2D(Sigma)
    V = V_matrix_4x4_uncoupled(alpha_x, alpha_y, beta_x, beta_y)
elif norm == '4D':
    U = np.array([[0, 1, 0, 0], [-1, 0, 0, 0], [0, 0, 0, 1], [0, 0, -1, 0]])
    eigvals, eigvecs = np.linalg.eig(np.matmul(Sigma, U))
    V = BL.construct_V(eigvecs)
Vinv = np.linalg.inv(V)
Sigma_n = np.linalg.multi_dot([Vinv, Sigma, Vinv.T])

In [ ]:
axes = myplt.rms_ellipses(Sigma, color='black', pad=0.75, alpha=0.15, fill=True, lw=0, zorder=0)
plt.suptitle('Reconstructed phase space')
reconstruction_lines(axes[2, 2], tmats_dict, moments_dict, plane='y-yp')
reconstruction_lines(axes[0, 0], tmats_dict, moments_dict, plane='x-xp',
                     legend=True, legend_kws=dict(loc=(1.15, 0)))
plt.savefig('_output/figures/corner.png', **savefig_kws)

In [ ]:
axes = myplt.rms_ellipses(Sigma_n, color='black', pad=0.75, alpha=0.15, fill=True, lw=0, zorder=0)
plt.suptitle('Reconstructed phase space (normalized {})'.format(norm))
reconstruction_lines(axes[2, 2], tmats_dict, moments_dict, plane='y-yp', norm_mat=V)
reconstruction_lines(axes[0, 0], tmats_dict, moments_dict, plane='x-xp', norm_mat=V,
                     legend=True, legend_kws=dict(loc=(1.15, 0)))
plt.savefig('_output/figures/corner_norm.png', **savefig_kws)

### Errors from LLSQ 

In [ ]:
def propagate_emittance_errors(Sigma, C):
    """Compute standard deviation from computed Sigma and LLSQ covariance matrix C.
    
    To do: do the same for the Twiss parameters. It should be easy.
    """
    eps_x, eps_y, eps_1, eps_2 = ba.emittances(Sigma)
    
    Cxx = C[:3, :3]
    Cyy = C[3:6, 3:6]
    Cxy = C[6:, 6:]
    
    grad_eps_x = (0.5 / eps_x) * np.array([Sigma[1, 1], Sigma[0, 0], -2 * Sigma[0, 1]])
    grad_eps_y = (0.5 / eps_y) * np.array([Sigma[3, 3], Sigma[2, 2], -2 * Sigma[2, 3]])
    eps_x_std = np.sqrt(np.linalg.multi_dot([grad_eps_x.T, Cxx, grad_eps_x]))
    eps_y_std = np.sqrt(np.linalg.multi_dot([grad_eps_y.T, Cyy, grad_eps_y]))
    
    g1 = np.linalg.det(Sigma)
    U = np.array([[0, 1, 0, 0], [-1, 0, 0, 0], [0, 0, 0, 1], [0, 0, -1, 0]])
    SigmaU = np.matmul(Sigma, U)
    g2 = np.trace(np.matmul(SigmaU, SigmaU))
    g = np.array(g1, g2)
    
    s11, s12, s13, s14 = Sigma[0, :]
    s22, s23, s24 = Sigma[1, 1:]
    s33, s34 = Sigma[2, 2:]
    s44 = Sigma[3, 3]

    grad_g = np.zeros((10, 2))
    grad_g[0, 0] = -s33*s24**2 + 2*s23*s24*s34 - s44*s23**2 + s22*(s33*s44 - s34**2)
    grad_g[1, 0] = -s33*s14**2 + 2*s13*s14*s34 - s44*s13**2 + s11*(s33*s44 - s34**2)
    grad_g[2, 0] = 2 * (s14*(s24*s33 - s23*s34) + s13*(-s24*s34 + s23*s44) + s12*(s34**2 - s33*s44))
    grad_g[3, 0] = -s22*s14**2 + 2*s12*s14*s24 - s44*s12**2 + s11*(s22*s44 - s24**2)
    grad_g[4, 0] = -s22*s13**2 + 2*s12*s13*s23 - s33*s12**2 + s11*(s22*s33 - s23**2)
    grad_g[5, 0] = 2 * (-s12*s14*s23 + s13*(s14*s22 - s12*s24) + s34*s12**2 + s11*(s23*s24 - s22*s34))
    grad_g[6, 0] = 2 * (s14*(-s23*s24 + s22*s34) + s13*(s24**2 - s22*s44) + s12*(-s24*s34 + s23*s44))
    grad_g[7, 0] = 2 * (s23*s14**2 - s14*(s13*s24 + s12*s34) + s12*s13*s44 + s11*(s24*s34 - s23*s44))
    grad_g[8, 0] = 2 * (s14*(s23**2 - s22*s33) + s13*(-s23*s24 + s22*s34) + s12*(s24*s33 - s23*s34))
    grad_g[9, 0] = 2 * (s24*s13**2 + s12*s14*s33 - s13*(s14*s23 + s12*s34) + s11*(-s24*s33 + s23*s34))

    grad_g[0, 1] = -2 * s22
    grad_g[1, 1] = -2 * s11
    grad_g[2, 1] = 4 * s12
    grad_g[3, 1] = -2 * s44
    grad_g[4, 1] = -2 * s33
    grad_g[5, 1] = 4 * s34
    grad_g[6, 1] = -4 * s24
    grad_g[7, 1] = 4 * s24
    grad_g[8, 1] = 4 * s23
    grad_g[9, 1] = -4 * s13
    
    Cg = np.linalg.multi_dot([grad_g.T, C, grad_g])
    
    H = np.sqrt(g2**2 - 16 * g1)
    deps1_dg1 = -1 / (eps_1 * H)
    deps2_dg1 = -1 / (eps_2 * H)
    deps1_dg2 = (1/8) * (1/eps_1) * (+g2/H - 1)
    deps2_dg2 = (1/8) * (1/eps_2) * (-g2/H - 1)
    grad_eps_1 = np.array([deps1_dg1, deps1_dg2])
    grad_eps_2 = np.array([deps2_dg1, deps2_dg2])
    
    eps_1_std = np.linalg.multi_dot([grad_eps_1.T, Cg, grad_eps_1])
    eps_2_std = np.linalg.multi_dot([grad_eps_2.T, Cg, grad_eps_2])

    return np.array([eps_x_std, eps_y_std, eps_1_std, eps_2_std])

In [ ]:
def llsq_cov_mat(A, res):
    n, p = np.shape(A)
    if n == p:
        C = np.linalg.inv(A)
    else:
        C = (res / (n - p)) * np.linalg.inv(np.matmul(A.T, A))
    return C

In [ ]:
def reconstruct(tmats_list, moments_list):
    # Form coefficient arrays and target arrays, 
    Axx, Ayy, Axy, bxx, byy, bxy = [], [], [], [], [], []
    for M, (sig_xx, sig_yy, sig_xy) in zip(tmats_list, moments_list):
        Axx.append([M[0, 0]**2, M[0, 1]**2, 2*M[0, 0]*M[0, 1]])
        Ayy.append([M[2, 2]**2, M[2, 3]**2, 2*M[2, 2]*M[2, 3]])
        Axy.append([M[0, 0]*M[2, 2],  M[0, 1]*M[2, 2],  M[0, 0]*M[2, 3],  M[0, 1]*M[2, 3]])
        bxx.append(sig_xx)
        byy.append(sig_yy)
        bxy.append(sig_xy)
    Axx = np.array(Axx)
    Ayy = np.array(Ayy)
    Axy = np.array(Axy)

    # Solve LLSQ problem.
    vec_xx, res_xx, _, _ = np.linalg.lstsq(Axx, bxx, rcond=None)
    vec_yy, res_yy, _, _ = np.linalg.lstsq(Ayy, byy, rcond=None)
    vec_xy, res_xy, _, _ = np.linalg.lstsq(Axy, bxy, rcond=None)
    if len(res_xy) == 0:
        res_xy = 0.0

#     # Use bounded solver for sig_xy since there are exactly four measurements.
#     def inverse_cholesky(x):
#         L = np.array([[x[0], 0, 0, 0],
#                       [x[1], x[2], 0, 0],
#                       [x[3], x[4], x[5], 0],
#                       [x[6], x[7], x[8], x[9]]])
#         return np.matmul(L, L.T)

#     def cost_func(x):
#         S = inverse_cholesky(x)
#         vec_xx = np.array([S[0, 0], S[1, 1], S[0, 1]])
#         vec_yy = np.array([S[2, 2], S[3, 3], S[2, 3]])
#         vec_xy = np.array([S[0, 2], S[1, 2], S[0, 3], S[1, 3]])
#         cost = 0.
#         cost += np.sum((np.matmul(Axx, vec_xx) - bxx)**2)
#         cost += np.sum((np.matmul(Ayy, vec_yy) - byy)**2)
#         cost += np.sum((np.matmul(Axy, vec_xy) - bxy)**2)
#         return cost

#     lb = -np.inf
#     ub = +np.inf
#     guess = np.random.uniform(-10, 10, size=10)
#     result = opt.minimize(cost_func, guess, bounds=opt.Bounds(lb, ub))
#     S = inverse_cholesky(result.x)

    
#     vec_xx = np.array([S[0, 0], S[1, 1], S[0, 1]])
#     vec_yy = np.array([S[2, 2], S[3, 3], S[2, 3]])
#     vec_xy = np.array([S[0, 2], S[1, 2], S[0, 3], S[1, 3]])
#     res_xx = np.sum((np.matmul(Axx, vec_xx) - bxx)**2)
#     res_yy = np.sum((np.matmul(Ayy, vec_yy) - byy)**2)
#     res_xy = np.sum((np.matmul(Axy, vec_xy) - bxy)**2)


    res_xx = float(res_xx)
    res_yy = float(res_yy)
    res_xy = float(res_xy)
    
    # Compute standard deviation of parameter vectors.
    Cxx = llsq_cov_mat(Axx, res_xx)
    Cyy = llsq_cov_mat(Ayy, res_yy)
    Cxy = llsq_cov_mat(Axy, res_xy)
    
    sig_11, sig_22, sig_12 = vec_xx
    sig_33, sig_44, sig_34 = vec_yy
    sig_13, sig_23, sig_14, sig_24 = vec_xy
    Sigma = np.array([[sig_11, sig_12, sig_13, sig_14],
                      [sig_12, sig_22, sig_23, sig_24],
                      [sig_13, sig_23, sig_33, sig_34],
                      [sig_14, sig_24, sig_34, sig_44]])
    C = np.zeros((10, 10))
    C[:3, :3] = Cxx
    C[3:6, 3:6] = Cyy
    C[6:, 6:] = Cxy
    
    return Sigma, C

In [ ]:
Sigma, C = reconstruct(tmats_list, moments_list)
eps_x, eps_y, eps_1, eps_2 = ba.emittances(Sigma)
eps_x_std, eps_y_std, eps_1_std, eps_2_std = propagate_emittance_errors(Sigma, C)

print('eps_x = {} +\- {} [mm mrad]'.format(eps_x, eps_x_std))
print('eps_y = {} +\- {} [mm mrad]'.format(eps_y, eps_y_std))
print('eps_1 = {} +\- {} [mm mrad]'.format(eps_1, eps_1_std))
print('eps_2 = {} +\- {} [mm mrad]'.format(eps_2, eps_2_std))

### Reconstruct at each step

In [ ]:
Sigmas = []
Cmats = []
for meas_index in range(n_meas):
    tmats_list, moments_list = [], []
    for ws_id in ws_ids:
        tmats_list.append(tmats_dict[ws_id][meas_index])
        moments_list.append(moments_dict[ws_id][meas_index])
    Sigma, C = reconstruct(tmats_list, moments_list)
    Sigmas.append(Sigma)
    Cmats.append(C)

In [ ]:
stats = ba.StatsReader()
stats.read_moments(np.array([Sigma[np.triu_indices(4)] for Sigma in Sigmas]))

In [ ]:
stds = np.array([propagate_emittance_errors(Sigma, C) for Sigma, C in zip(Sigmas, Cmats)])

In [ ]:
fig, ax = pplt.subplots(figsize=(3.5, 2.5))
plt_kws = dict(marker='.')
n_turns = list(range(50, 550, 50))
g1 = ax.errorbar(n_turns, stats.twiss2D['eps_x'], yerr=stds[:, 0], **plt_kws)
g2 = ax.errorbar(n_turns, stats.twiss2D['eps_y'], yerr=stds[:, 1], **plt_kws)
g3 = ax.errorbar(n_turns, stats.twiss4D['eps_1'], yerr=stds[:, 2], **plt_kws)
# g4 = ax.errorbar(n_turns, stats.twiss4D['eps_2'], yerr=stds[:, 3], **plt_kws)
g4 = ax.errorbar(n_turns, stats.twiss4D['eps_2'], **plt_kws)
ax.legend([g1, g2, g3, g4], eps_labels, loc='r', ncols=1)
ax.format(xlabel='Number of injected turns', ylabel='[mm mrad]')
ax.grid(axis='y')
plt.savefig('_output/figures/emittances.png', **savefig_kws)

In [ ]:
fig, ax = pplt.subplots(figsize=(3.5, 2.5))
plt_kws = dict(marker='.')
n_turns = list(range(50, 550, 50))
g1 = ax.plot(n_turns, stats.twiss2D['eps_x'], **plt_kws)
g2 = ax.plot(n_turns, stats.twiss2D['eps_y'], **plt_kws)
g3 = ax.plot(n_turns, stats.twiss4D['eps_1'], **plt_kws)
g4 = ax.plot(n_turns, stats.twiss4D['eps_2'], **plt_kws)
ax.legend(handles=[g1, g2, g3, g4], labels=eps_labels, loc='r', ncols=1)
ax.format(xlabel='Number of injected turns', ylabel='[mm mrad]')
ax.grid(axis='y')
plt.savefig('_output/figures/emittances.png', **savefig_kws)

In [ ]:
fig, ax = pplt.subplots(figsize=(3.5, 2.5))
g1 = ax.plot(n_turns, stats.twiss2D['eps_x'] * stats.twiss2D['eps_y'], color='red', **plt_kws)
g2 = ax.plot(n_turns, stats.twiss4D['eps_1'] * stats.twiss4D['eps_2'], color='blue', **plt_kws)
ax.legend(handles=[g1, g2], labels=[''.join(eps_labels[:2]), ''.join(eps_labels[2:])], loc='r', ncols=1)
ax.format(xlabel='Number of injected turns', ylabel=r'[mm$^2$ mrad$^2$]')
ax.grid(axis='y')
plt.savefig('_output/figures/emittances4D.png', **savefig_kws)

In [ ]:
fig, axes = pplt.subplots(nrows=2, figsize=(3.5, 3.5), spany=False, aligny=True)
g1 = axes[0].plot(n_turns, stats.twiss2D['eps_x'], **plt_kws)
g2 = axes[0].plot(n_turns, stats.twiss2D['eps_y'], **plt_kws)
g3 = axes[0].plot(n_turns, stats.twiss4D['eps_1'], **plt_kws)
g4 = axes[0].plot(n_turns, stats.twiss4D['eps_2'], **plt_kws)
axes[0].legend(handles=[g1, g2, g3, g4], labels=eps_labels, loc='r', ncols=1)

g1 = axes[1].plot(n_turns, stats.twiss2D['eps_x'] * stats.twiss2D['eps_y'], color='red', **plt_kws)
g2 = axes[1].plot(n_turns, stats.twiss4D['eps_1'] * stats.twiss4D['eps_2'], color='blue', **plt_kws)
axes[1].legend(handles=[g1, g2], labels=[''.join(eps_labels[:2]), ''.join(eps_labels[2:])], 
               loc='r', ncols=1)

axes[0].format(ylabel='[mm mrad]', xlabel='Turn number')
axes[1].format(ylabel=r'[mm$^2$ mrad$^2$]')
for ax in axes:
    ax.grid(axis='y')
plt.savefig('_output/figures/emittances_combined.png', facecolor='w', dpi=300)

In [ ]:
eps_x = stats.twiss2D['eps_x']
eps_y = stats.twiss2D['eps_y']
eps_1 = stats.twiss4D['eps_1']
eps_2 = stats.twiss4D['eps_2']
C = 1.0 - (eps_1 * eps_2) / (eps_x * eps_y)

fig, ax = pplt.subplots(figsize=(2.5, 2.5))
ax.plot(n_turns, C, marker='.', color='k')
ax.grid(axis='y')
ax.format(xlabel='Turn number', 
          title=r'C = 1 - $\frac{\varepsilon_1\varepsilon_2}{\varepsilon_x\varepsilon_y}$',
          ylim=(0, 1))
plt.savefig('_output/figures/C.png', **savefig_kws)

In [ ]:
# for meas_index in meas_indices:

#     Sigma = Sigmas[meas_index]
#     axes = myplt.rms_ellipses(Sigma, color='black', pad=0.75, alpha=0.15, fill=True, lw=0, zorder=0)
#     plt.suptitle('Reconstructed phase space - turn {}'.format(n_turns[meas_index]), fontsize='medium')

#     _tmats_dict = dict()
#     _moments_dict = dict()
#     for ws_id in ws_ids:
#         _tmats_dict[ws_id] = tmats_dict[ws_id][meas_index:meas_index + 1]
#         _moments_dict[ws_id] = moments_dict[ws_id][meas_index:meas_index + 1]
#     reconstruction_lines(axes[2, 2], _tmats_dict, _moments_dict, plane='y-yp')
#     reconstruction_lines(axes[0, 0], _tmats_dict, _moments_dict, plane='x-xp',
#                          legend=True, legend_kws=dict(loc=(1.15, 0)))
#     plt.savefig('_output/figures/corner_ninjturns={}.png'.format(n_turns[meas_index]), **savefig_kws)
#     plt.show()

In [ ]:
cmap = pplt.Colormap('flare')

axes = myplt.rms_ellipses(Sigmas, 
                          cmap=cmap
                         )
plt.savefig('_output/figures/corner_all.png', **savefig_kws)

## Target image 

In [ ]:
from target_image.target_image_analysis import Image
from target_image.target_image_analysis import PIXEL_WIDTH
from target_image import target_image_analysis
from plotting import zoom_axis
from plotting import JointGrid

In [ ]:
path = os.path.join(folder, 'target_images')
filenames = utils.list_files(path)
images = target_image_analysis.read_files(filenames)

In [ ]:
import time
start_time = time.time()

height = 4.0
ratio = 4.0
zoom = 1.25
joint_kws = dict(shading='flat', cmap='dusk_r')
marginal_kws = dict(color='k')

for hist in [False, True]:        
    for same_ylim in [True, False]:
        for sigma in [0.0, 1.5]:
            hmax = vmax = None
            if same_ylim:
                hmax = vmax = 0.
                for image in images:
                    Z = image.filter(sigma)
                    hmax = max(hmax, np.max(np.sum(Z, axis=0)))
                    hmax = max(hmax, np.max(np.sum(Z, axis=1)))
                    vmax = max(vmax, np.max(Z))
            for i, image in enumerate(images):
                if hist:
                    grid = JointGrid(height=height, ratio=ratio)
                    grid.ax_joint.set_xlabel('x [mm]', fontsize=9)
                    grid.ax_joint.set_ylabel('y [mm]', fontsize=9)
                    grid.ax_joint.annotate('x [mm]', xy=(0.43, -0.18), xycoords='axes fraction', fontsize=9)
                    grid.plot_image(image.xx, image.yy, image.filter(sigma), vmax=vmax, hmax=hmax, joint_kws=joint_kws, marginal_kws=marginal_kws)
                    grid.zoom(zoom)
                    grid.ax_joint.annotate(r'Filter $\sigma$ = {}'.format(sigma), xy=(0.01, 0.94), xycoords='axes fraction', color='white', fontsize='small')
                    grid.ax_marg_x.set_title('n_turns = {}'.format(n_turns[i]))  
                    ax = grid.ax_joint
                else:
                    fig, ax = pplt.subplots(figsize=(height, height), aspect=1)
                    ax.pcolormesh(image.X, image.Y, image.filter(sigma), vmax=vmax, **joint_kws)
                    ax.format(xlabel='x [mm]', ylabel='y [mm]')
                    zoom_axis(ax, zoom)
                angle, cx, cy = ba.rms_ellipse_dims(Sigmas[i])
                image.fit_gauss2d(use_filtered=True)
                myplt.ellipse(ax, cx, cy, angle, center=(image.mean_x, image.mean_y), color='red8', lw=0.4)        
                figname = '_output/figures/targetimage_hist={}_sameylim={}_sigma{}_turn{}.png'.format(hist, same_ylim, sigma, n_turns[i])
                plt.savefig(figname, **savefig_kws)
                plt.close()
                
print('Runtime = {} [seconds]'.format(time.time() - start_time))